In [5]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

import pandas as pd
import tensorflow as tf

from tensorflow import keras as k
from utility import *

In [6]:
data = load_data()
X, y = create_XY(data)

enc = OneHotEncoder(sparse=False)
y = enc.fit_transform(y)
X_imputed = SimpleImputer().fit_transform(X)

# LSTM

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_imputed, y, shuffle=False, test_size=test_size)

In [ ]:
x_train = reshape_to_inputshape(x_train, trn_ssn)
y_train = reshape_to_inputshape(y_train, trn_ssn)

x_test = reshape_to_inputshape(x_test, tst_ssn)
y_test = reshape_to_inputshape(y_test,  tst_ssn)

In [ ]:
model = k.models.Sequential(
    [
        k.layers.LSTM(64, return_sequences=True),
        k.layers.Dropout(0.4),
        k.layers.Dense(1000, activation="relu"),
        k.layers.Dropout(0.3),
        k.layers.Dense(250, activation="relu"),
        k.layers.Dropout(0.2),
        k.layers.Dense(3, activation="softmax")
    ]
)

print(model(x_train).shape)
model.summary()

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.01),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=6)

model.fit(x_train, y_train, epochs=500, callbacks=[callback])

In [ ]:
model.evaluate(x_test, y_test)
report2(model, x_train, y_train, "train", enc)
report2(model, x_test, y_test, "test", enc)

## SimpleRNN

In [ ]:
model = k.models.Sequential(
    [
        k.layers.SimpleRNN(64, return_sequences=True),
        k.layers.Dropout(0.4),
        k.layers.Dense(1000, activation="relu"),
        k.layers.Dropout(0.3),
        k.layers.Dense(250, activation="relu"),
        k.layers.Dropout(0.2),
        k.layers.Dense(3, activation="softmax")
    ]
)

print(model(x_train).shape)
model.summary()

In [ ]:

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.01),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=6)

model.fit(x_train, y_train, epochs=500, callbacks=[callback])
model.evaluate(x_test, y_test)
report2(model, x_train, y_train, "train", enc)
report2(model, x_test, y_test, "test", enc)

## Bidirectional

In [ ]:
model = k.models.Sequential(
    [
        k.layers.Bidirectional(k.layers.LSTM(64, return_sequences=True)),
        k.layers.Dropout(0.4),
        k.layers.Dense(1000, activation="relu"),
        k.layers.Dropout(0.3),
        k.layers.Dense(250, activation="relu"),
        k.layers.Dropout(0.2),
        k.layers.Dense(3, activation="softmax")
    ]
)

print(model(x_train).shape)
model.summary()

In [ ]:

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.01),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=6)

model.fit(x_train, y_train, epochs=500, callbacks=[callback])
model.evaluate(x_test, y_test)
report2(model, x_train, y_train, "train", enc)
report2(model, x_test, y_test, "test", enc)

## 2 LSTM concatenate + dense e dropout in sequenza

In [ ]:
inputs = k.layers.Input(shape=(10,94))
lstm1 = k.layers.LSTM(100, return_sequences=True, activation="relu")(inputs)
lstm2 = k.layers.LSTM(50, return_sequences=True, activation="relu")(inputs)
concatenated = k.layers.Concatenate()([
    lstm1,
    lstm2
])

out = k.layers.Dropout(0.5)(concatenated)
out = k.layers.Dense(1000, activation="relu")(out)
out = k.layers.Dropout(0.5)(out)
out = k.layers.Dense(250, activation="relu")(out)
out = k.layers.Dropout(0.5)(out)
out = k.layers.Dense(3, activation="softmax")(out)

model = k.models.Model(inputs=inputs, outputs=out)

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.01),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=6)

model.fit(x_train, y_train, epochs=500, callbacks=[callback])

In [ ]:
model.evaluate(x_test, y_test)
report2(model, x_train, y_train, "train", enc)
report2(model, x_test, y_test, "test", enc)

### Fit
loss: 0.9373 - accuracy: 0.5609
### Evaluate
loss: 0.9192 - accuracy: 0.5930
### Classification report

| | precision | recall | f1-score | support |
| - | ----------- | ------ | -------- | ------- |
| A | 0.609 | 0.614 | 0.612 | 345 |
| D | 0.321 | 0.201 | 0.247 | 254 |
| H | 0.652 | 0.763 | 0.704 | 541 |
| accuracy | |  | 0.593 | 1140 |
| macro avg | 0.527 | 0.526 | 0.521 | 1140 |
| weighted avg | 0.565 | 0.593 | 0.574 | 1140 |

## LSTM in parallelo

In [ ]:
inputs = k.layers.Input(shape=(10,94))
x = []
Tx = 10
for t in range(Tx):
    module = k.layers.LSTM((t+1)*10, return_sequences=True)(inputs)
    module = k.layers.Dropout(0.7)(module)
    x.append(module)
x = k.layers.Concatenate()(x)

out = k.layers.Dense(1000, activation="relu")(x)
out = k.layers.Dropout(0.7)(out)
out = k.layers.Dense(250, activation="relu")(out)
out = k.layers.Dense(3, activation="softmax")(out)

txLstm = k.models.Model(inputs=inputs, outputs=out)

In [ ]:
txLstm.summary()

In [ ]:
txLstm.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.01),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=6)

txLstm.fit(x_train, y_train, epochs=500, callbacks=[callback], batch_size=16)

In [ ]:
txLstm.evaluate(x_test, y_test)
report2(txLstm, x_train, y_train, "train", enc)
report2(txLstm, x_test, y_test, "test", enc)

# GRU

In [ ]:
gru = k.models.Sequential([
    k.layers.GRU(100, return_sequences=True, activation="relu"),
    k.layers.Dropout(0.4),
    k.layers.Dense(1000, activation="relu"),
    k.layers.Dropout(0.3),
    k.layers.Dense(250, activation="relu"),
    k.layers.Dropout(0.2),
    k.layers.Dense(3, activation="softmax")
])

In [ ]:
gru(x_train)
gru.summary()

In [ ]:
gru.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.01),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=6)

gru.fit(x_train, y_train, epochs=500, callbacks=[callback])

In [ ]:
gru.evaluate(x_test, y_test)
report2(gru, x_train, y_train, "train", enc)
report2(gru, x_test, y_test, "test", enc)

# Time distributed

In [ ]:
def time_step(data, label, step=10):
    x,y = [],[]
    for i in range(data.shape[0] - step):
        x.append(data[i:i+step])
        y.append(label[i+step])
    return np.array(x), np.array(y)

In [ ]:
def create_dataset(team_name):
    ht = data.loc[data['HomeTeam']==team_name]
    at = data.loc[data['AwayTeam']==team_name]
    team_data = pd.concat([ht,at])
    team_data['Team'] = team_name
    team_data_label = team_data['FTR']
    team_data_featured = pd.get_dummies(team_data[features])
    return team_data_featured,team_data_label

In [ ]:
team_name = 'Chelsea'
data_f, data_l = create_dataset(team_name)
data_f = SimpleImputer().fit_transform(data_f)
x_ars, y_ars = time_step(data_f, data_l.to_numpy())

print(x_ars.shape)
print(y_ars.shape)

y_ars = enc.fit_transform(y_ars.reshape(-1,1))
x_train, x_test, y_train, y_test = train_test_split(x_ars, y_ars, shuffle=False, test_size=test_size)

Tx = x_train.shape[1] #Time steps
Ty = y_train.shape[1] #Time Steps

In [ ]:
teams = set(data['HomeTeam'])
df_x = pd.DataFrame()
df_y = pd.DataFrame()
for team in teams:
    data_f, data_l = create_dataset(team)
    #data_f = SimpleImputer().fit_transform(pd.get_dummies(data_f))  
    x_team, y_team = time_step(data_f, data_l.to_numpy())
    y_team = enc.fit_transform(y_team.reshape(-1,1))
    df_x = pd.concat([df_x, pd.DataFrame(x_team.reshape(x_team.shape[0], -1))])
    df_y = pd.concat([df_y, pd.DataFrame(y_team)])
df_x = SimpleImputer().fit_transform(pd.get_dummies(df_x))  
df_x = df_x.reshape(df_x.shape[0], 10, -1)

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, shuffle=False, test_size=test_size)

In [ ]:
model = k.models.Sequential(
    [
        k.layers.GRU(128, dropout=0.5),
        k.layers.Flatten(),
        k.layers.Dense(40, activation="relu"),
        k.layers.Dropout(0.4),
        k.layers.Dense(20, activation="relu"),
        k.layers.Dropout(0.2),
        k.layers.Dense(10, activation="relu"),
        k.layers.Dense(3, activation="softmax")
    ]
)

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=["accuracy"]
)
model(x_train)
model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=6)

model.fit(x_train, y_train, epochs=100, callbacks=[callback])

In [ ]:
model.evaluate(x_test, y_test)
print(y_test.shape)
report1(model, x_train, y_train, "train", enc)
report1(model, x_test, y_test, "test", enc)

# LSTM timestep = features

In [ ]:
features_not_teams = ['HTeamEloScore', 'ATeamEloScore', 
            'HTdaysSinceLastMatch', 'ATdaysSinceLastMatch', 
            'HTW_rate', 'ATW_rate', 'ATD_rate', 'HTD_rate', 
            '7_HTW_rate', '12_HTW_rate', '7_ATW_rate', '12_ATW_rate', 
            '7_HTD_rate', '12_HTD_rate', '7_ATD_rate', '12_ATD_rate',
            '7_HTL_rate', '12_HTL_rate', '7_ATL_rate', '12_ATL_rate',
            '5_HTHW_rate', '5_ATAW_rate']

X = pd.get_dummies(data[features_not_teams]).to_numpy()
y = data[['FTR']].to_numpy().ravel().reshape(-1, 1)
enc = OneHotEncoder(sparse=False)
y = enc.fit_transform(y)
X_imputed = SimpleImputer().fit_transform(X)

In [8]:
def reshape_features(input: np.ndarray):
    arr = input.reshape(input.shape[0], input.shape[1], 1)
    return arr

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_imputed, y, shuffle=False, test_size=test_size)

x_train = reshape_features(x_train)
x_test = reshape_features(x_test)

print("x_train:", x_train.shape)
print("x_test:", x_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

In [ ]:
model = k.models.Sequential(
    [
        k.layers.LSTM(64),
        k.layers.Flatten(),
        k.layers.Dense(40, activation="relu"),
        k.layers.Dropout(0.4),
        k.layers.Dense(20, activation="relu"),
        k.layers.Dropout(0.2),
        k.layers.Dense(10, activation="relu"),
        k.layers.Dense(3, activation="softmax")
    ]
)

model(x_train)
model.summary()

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.01),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

model.fit(x_train, y_train, epochs=500, callbacks=[callback])

In [ ]:
model = k.models.Sequential(
    [
        k.layers.Bidirectional(k.layers.LSTM(64)),
        k.layers.Dense(3, activation="softmax")
    ]
)

model(x_train)
model.summary()

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.01),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

model.fit(x_train, y_train, epochs=10, callbacks=[callback])

In [ ]:
report1(model, x_train, y_train, 'train', enc)
report1(model, x_test, y_test, 'test', enc)

# WaveNet

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X_imputed, y, shuffle=False, test_size=test_size)

x_train = reshape_features(x_train)
x_test = reshape_features(x_test)

print("x_train:", x_train.shape)
print("x_test:", x_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

x_train: (3040, 94, 1)
x_test: (1140, 94, 1)
y_train: (3040, 3)
y_test: (1140, 3)


In [10]:
model = k.models.Sequential()

for i in [1, 2, 4]:
    model.add(k.layers.Conv1D(2**(4 + i), 2, dilation_rate=i, padding="causal"))
    model.add(k.layers.BatchNormalization())
    model.add(k.layers.Activation(activation="relu"))
    model.add(k.layers.MaxPool1D(pool_size=2, strides=1, padding="same"))

# model.add(k.layers.Conv1D(3, 2, dilation_rate=1, padding="same"))
# model.add(k.layers.GlobalAveragePooling1D())
model.add(k.layers.Flatten())
model.add(k.layers.Dense(100))
model.add(k.layers.Dense(3, activation="softmax"))

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-12-22 12:39:38.108214: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-22 12:39:38.108356: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=["accuracy"]
)
model(x_train)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (3040, 94, 32)            96        
_________________________________________________________________
batch_normalization (BatchNo (3040, 94, 32)            128       
_________________________________________________________________
activation (Activation)      (3040, 94, 32)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (3040, 94, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (3040, 94, 64)            4160      
_________________________________________________________________
batch_normalization_1 (Batch (3040, 94, 64)            256       
_________________________________________________________________
activation_1 (Activation)    (3040, 94, 64)            0

In [12]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

model.fit(x_train, y_train, epochs=1000, batch_size=64, callbacks=[callback])

2022-12-22 12:41:47.229128: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-12-22 12:41:47.229352: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/1000


2022-12-22 12:41:47.898998: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


48/48 [==============================] - 5s 62ms/step - loss: 5.0365 - accuracy: 0.4678
Epoch 2/1000
48/48 [==============================] - 3s 55ms/step - loss: 1.9083 - accuracy: 0.4618
Epoch 3/1000
48/48 [==============================] - 3s 55ms/step - loss: 1.5799 - accuracy: 0.4599
Epoch 4/1000
48/48 [==============================] - 3s 54ms/step - loss: 1.5360 - accuracy: 0.4684
Epoch 5/1000
48/48 [==============================] - 3s 55ms/step - loss: 1.5011 - accuracy: 0.4849
Epoch 6/1000
48/48 [==============================] - 3s 56ms/step - loss: 1.0998 - accuracy: 0.5039
Epoch 7/1000
48/48 [==============================] - 3s 54ms/step - loss: 1.0663 - accuracy: 0.5204
Epoch 8/1000
48/48 [==============================] - 3s 55ms/step - loss: 1.1309 - accuracy: 0.4961
Epoch 9/1000
48/48 [==============================] - 3s 54ms/step - loss: 1.1000 - accuracy: 0.5069
Epoch 10/1000
48/48 [==============================] - 3s 54ms/step - loss: 1.0228 - accuracy: 0.5368
Ep

In [13]:
report1(model, x_train, y_train, "train", enc)
report1(model, x_test, y_test, "test", enc)

2022-12-22 12:44:58.466544: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


TRAIN REPORT
              precision    recall  f1-score   support

           A      0.628     0.537     0.579       867
           D      0.499     0.249     0.332       783
           H      0.622     0.853     0.719      1390

    accuracy                          0.608      3040
   macro avg      0.583     0.547     0.544      3040
weighted avg      0.592     0.608     0.580      3040

--------------------------------------------------


2022-12-22 12:44:59.779101: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


TEST REPORT
              precision    recall  f1-score   support

           A      0.587     0.646     0.615       345
           D      0.286     0.102     0.151       254
           H      0.619     0.765     0.684       541

    accuracy                          0.582      1140
   macro avg      0.497     0.505     0.483      1140
weighted avg      0.535     0.582     0.544      1140

--------------------------------------------------
